In [271]:
import math
import functools

import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats

import seaborn as sns
import matplotlib as plt

from IPython.display import display, Latex

In [ ]:
# обозначения переменных:
# n - объём выборки
# m - количество предикторов
# num, denom - числитель и знаменатель в дробных выражения соответственно

In [44]:
df = pd.read_csv('IB_statistics_data_hw2.4.txt', sep=' ')
df.head()

,Ex3.y,Ex3.x1,Ex3.x2,Ex3.x3,Ex3.x4,Ex3.x5,Ex5.y,Ex5.x1,Ex5.x2,Ex6.y,Ex6.x1,Ex6.x2
0,28.872405,4,7,29,26,-8.886853,23453.016482,0.824387,2.686985,5.434036,-2.266430,2.411750
1,9.293858,2,10,34,26,-10.569351,0.135840,0.842774,3.828651,1.532948,1.989858,0.625990
2,32.019721,4,8,32,28,-8.923073,8.828238,1.523876,1.636613,1.463174,-4.500657,0.595529
3,43.993442,4,6,26,24,-6.860549,29.017670,0.799176,1.749826,3.012252,0.629619,1.429877
4,14.779713,2,6,22,18,-7.219951,2118.591582,0.969108,2.278681,1.140469,0.152484,0.643395


# 1.
Пример - выборка из 10-мерного  нормального распределения с единичной ковариационной матрицей

In [31]:
def R2adj(Y_pred, Y_true, n, m):
    num = (n - 1) * np.sum((Y_true - Y_pred)**2)
    denom = (n - m) * np.sum((Y_true - Y.mean())**2)
    return 1 - num / denom 

In [36]:
# генерация выборки
# (m + 1) - мерное нормальное распределение с единичной ковариационной матрицей
# вектор средних равен (0, 1, ... m)

n = 200
m = 10
sample = pd.DataFrame(sp.stats.multivariate_normal(np.arange(m + 1), cov=1).rvs(n))
X, Y = sample.iloc[:, 1:], sample.iloc[:, 0]
X.columns = [f'x{i}' for i in range(1, m+1)]
X.insert(0, 'x0', 1)

# МНК-оценка 
beta = np.linalg.lstsq(X, Y, rcond=None)[0]

display(Latex(r'$R^2_{adj} =' + f' {round(R2adj(X @ beta, Y, n, m), 3)}$'))

<IPython.core.display.Latex object>

# 2.

Доказать: в задаче
$||Ax - b|| \rightarrow \min$
минимум достигается при $x = A^{(1, 3)} b$

Доказательство:
$$
A : \mathbb R^n \rightarrow \mathbb R^m; \\
b = \text{Pr}_{ImA}b + \text{Pr}_{ImA^\perp}b = \text{Pr}_{ImA}b + \text{Pr}_{KerA^T} b\\
b - Ax = (\text{Pr}_{ImA} b - Ax) + \text{Pr}_{KerA^T} b \\
\Rightarrow ||Ax - b ||^2 = ||Ax - \text{Pr}_{ImA} b||^2 + ||\text{Pr}_{KerA^T} b||^2 
$$
Минимум $||Ax - b ||^2$ достигается при условии $Ax = \text{Pr}_{ImA} b$. 

Т.е. нужно доказать, что 
$$
A A^{(1, 3)}b = \text{Pr}_{ImA} b 
$$

Это следует из того, что:
$$
1.\ \ (A A^{(1,3)}A) A^{(1,3)} = A A^{(1,3)} \\
2. \ \ \forall x:\ \ AA^{(1,3)} x \Rightarrow (A A^{(1,3)})^T x = 0 \Rightarrow (A^{(1,3)})^T A^T x 
\Rightarrow A^T x = 0 \Rightarrow Ker AA^{(1,3)} = Ker A^T
$$
$\blacksquare$


# 3.

In [405]:
X = df.loc[:, 'Ex3.x1':'Ex3.x5']
X.columns = [f'x{i}' for i in range(1, 6)]
X.insert(0, 'x0', 1)

Y = df['Ex3.y']

### a)

Условие можно переписать как $H \beta = 0,\ H = (0, 1, 1, 1, 1, 0)$.

Нужно отсыкать матрицу Z, такую, что 
$$
H (X^+ + (1 - X^+ X)Z)Y = 0
$$


In [490]:
beta_ols = np.linalg.lstsq(X, Y, rcond=None)[0]

X = np.array(X)
Y = np.array(Y)

Xpinv = np.linalg.pinv(X)

XpinvX = Xpinv @ X
identity = np.eye(XpinvX.shape[0])

H = np.array([0, 1, 1, 1, 1, 0])

def equation(Z):
    return H @ (Xpinv + (identity - XpinvX) @ Z) @ Y

Zinit = np.ones(shape=X.T.shape)

print(H @ (Xpinv + (identity - XpinvX) @ Zinit) @ Y)

sp.optimize.fsolve(equation, Zinit)

6314.662336243964


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1440 is different from 6)

почему-то в fsolve то же выражение, которое снаружи выполняется нормально,
падает из-за несовпадений размерности...

### b) 
$||\beta|| \rightarrow \min.$

По теореме об оптимальных свойствах псевдообратной матрицы, решением с минимальной нормой является
$\beta = X^+ Y$

In [404]:
beta = np.linalg.pinv(X) @ Y
display(Latex(r'$\hat {\beta} =$'))
print(*[round(b, 3) for b in beta], sep='\n')

<IPython.core.display.Latex object>

12.068
2.609
-2.107
-1.103
3.614
3.984


### c)

### d)

# 4.

# 5.

Преобразуем модель:

$$
y = e^{\beta_0 \varepsilon} \cdot x_1^{\beta_1} \cdot x_2^{\beta_2} \\
\Rightarrow \ln y = \beta_0 \varepsilon + \beta_1 \ln x_1 + \beta_2\ln x_2 = \\
= \varepsilon' + \beta_1 \ln x_1 + \beta_2\ln x_2
$$
$\beta_0$ "уходит" в дисперсию ошибки. Соответственно можно выбрать произвольное значение $\beta_0$, тогда дисперсия ошибки соответствующим образом изменится.

### Построение ОМП оценки
Т.к. ошибки распределены нормально, то МНК-оценка будет совпадать с ОМП оценкой.

In [345]:
X = df[['Ex5.x1', 'Ex5.x2']]
X.columns = ['x1', 'x2']
Y = df['Ex5.y']


# преобразование модели
X = np.log(np.array(X))
Y = np.log(np.array(Y))

# МНК оценка в преобразованной модели
hat_beta = np.linalg.lstsq(X, Y, rcond=None)[0]
# Добавляем beta_0 = 1
hat_beta = np.insert(hat_beta, 0, 1) 
display(Latex(r'$\hat \beta = $'))
print(*[round(b, 3) for b in beta])

<IPython.core.display.Latex object>

1.0 1.151 2.794


### Проверка гипотезы
Воспользуемся "грубым" методом, проверив гипотезу для линеаризованной модели.

Доверительная область:
$
\left\{ x: (\hat \beta - x)^T (X^T X) (\hat \beta - x) < \frac{m}{n-m}s^2 F_{m, n - m}(0.95) \right\}
$

Т.к. $\beta_0$ можно выбрать любым, то проверим для вектора из двух оставшихся компонент.

In [364]:
GAMMA = 0.95

beta0 = np.array([1, 1])
beta = hat_beta[1:]

n, m = X.shape

# МНК оценка для sigma2
s2 = np.sum((Y - X @ beta)**2) / (n - m)

# 0.95-квантиль распределения F(m, n - m)
distr = sp.stats.f(m, n - m)
f95 = sp.optimize.fsolve(lambda x: distr.cdf(x) - 0.95, 10)[0]

# граница доверительной области
bound = m / (n - m) * s2 * f95

print('Граница доверительной области: ', round(bound, 3))

# лежит ли beta (1, 1) в доверительной области
T = (beta - beta0).T @ (X.T @ X) @ (beta - beta0)
print('Значение статистики: ', round(T, 3))

Граница доверительной области:  0.249
Значение статистики:  646.483


$\Rightarrow H_0$ отклоняется.

# 6.

### 1) ОМП-оценка
Т.к. ошибки распределны нормально, то ОМП оценка совпадает с МНК оценкой.

In [286]:
X = df[['Ex6.x1', 'Ex6.x2']]
X.columns = ['x1', 'x2']
Y = df['Ex6.y']
X = np.array(X)
Y = np.array(Y)

def loss(beta):
    # функция потерь - сумма квадратов отклонений
    beta1, beta2 = beta
    
    s = 0
    for i, row in enumerate(X):
        x1, x2 = row
        y_true = Y[i]
        y_pred = beta1 * x1 * x2 + math.e **(beta2 * x2)
        s += (y_true - y_pred)**2
    
    return s

initial_beta = [0, 1]
hat_beta = scipy.optimize.minimize(loss, initial_beta).x
display(Latex(r'$\hat \beta = $'))
hat_beta 

<IPython.core.display.Latex object>

array([0.00875047, 0.69356228])

### 2) Проверка гипотезы

$h(\beta)$ - отображение из $\mathbb R^p$ в $\mathbb R^q$.

$H_0: h(\beta) = 0$ против альтернативы $H_1: h(\beta) \ne 0$

В случае нормальных ошибок (и единственности МНК оценки) применим следующий критерий (источник -A. Ronald Gallant - Nonlinear Statistical Models):

$\hat \beta = \text{argmin}\ \text{SSE}(\beta) = \text{argmin} \ (y - f(\beta))^T(y - f(\beta))$;

$\text{SSE}_{full} = \text{SSE}(\hat \beta)$;

$\tilde \beta = \text{argmin}\ \text{SSE}(\beta)$ при условии $h(\beta) = 0$;

$\text{SSE}_{reduced} = \text{SSE}(\tilde \beta)$

Статистика:
$$
L = \frac{(\text{SSE}_{reduced} - \text{SSE}_{full}) /q}{\text{SSE}_{full} / (n - p)} \sim F(q, n - p)
$$
($p$ - число регрессоров).

В нашем случае 
$$
h(\beta) = 0 \Leftrightarrow \begin{cases}
\beta_1 = 0, \\
\beta_2 = \log 2
\end{cases}
$$
Т.е. $q = 2$;

$\tilde \beta = \beta_0 = (0, \log 2)$

$\hat \beta$ посчитана в предыдущем пункте.

In [303]:
ALPHA = 0.05

beta0 = np.array([0, np.log(2)])
n, p = X.shape
q = 2

SSE_full = (X @ hat_beta).T @ (X @ hat_beta)
SSE_reduced = (X @ beta0).T @ (X @ beta0)

num = (SSE_reduced - SSE_full) / q
denom = SSE_full / (n - p)
L = num / denom
display(Latex(f'$L = {round(L, 3)}$'))


# Критическое значение
L_crit = sp.optimize.fsolve(lambda x: sp.stats.f(q, n - p).cdf(x) - (1 - ALPHA), 0)[0]
display(Latex(r'$L_{cr} = %s$' % round(L_crit, 3)))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Значение меньше критического $\Rightarrow$ $H_0$ не отклоняется.